In [22]:
import optuna
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_contour


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.metrics import roc_auc_score


from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn import metrics
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import mlflow
import utils
from sklearn.model_selection import train_test_split, StratifiedKFold

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import operator





from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler



PATH = '/Users/mimi/Desktop/PYTHON/TURING/HOME/MachLearning/S2/data/archive (1).zip'

df = pd.read_csv(PATH)
df.drop(columns='id', inplace=True)
df_ = df.copy()
df.drop(df[df.gender == 'Other'].index,inplace=True)
df.reset_index(drop=True,inplace=True)





df.head()
X = df.drop(columns='stroke')
y = df['stroke']



In [25]:
X_transformed = utils.preprocess_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)
classifier = LogisticRegression()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
y_pred_proba = classifier.predict_proba(X_test)[:, 1]  


acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred,average='micro')
recall = recall_score(y_test, y_pred,average='micro')
roc_auc = roc_auc_score(y_test, y_pred_proba)  


metrics = {'accuracy': round(acc, 3), 
           'precision': round(prec, 3), 
           'recall': round(recall, 3), 
           'roc_auc': round(roc_auc, 3)}

coef = classifier.coef_
intercept = classifier.intercept_
params = {'coef':coef, 'intercept': intercept}
roc_auc = roc_auc_score(y_test, y_pred_proba)



mlflow.set_tracking_uri("http://localhost:5000")
experiment_name = "stroke_classifier"
run_name="stroke_classifier_ver1.0"



mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    #mlflow.log_artifact('./scatter.png')
    mlflow.sklearn.log_model(classifier, "model")
print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

2024/04/07 18:17:47 INFO mlflow.tracking.fluent: Experiment with name 'stroke_classifier' does not exist. Creating a new experiment.


Run - stroke_classifier_ver1.0 is logged to Experiment - stroke_classifier


In [ ]:
download_path = "/Users/mimi/Desktop/PYTHON/TURING/HOME/MachLearning/S2/mlruns/models" 
client.download_artifacts(run_id = run_id, path="scatter.png", dst_path = download_path)

In [ ]:
with mlflow.start_run(run_name = " Log Regression Experiment") as run:
  # Run LinearRegression model
  ridge = Ridge(alpha = 0.4, normalize = True)
  
  # Naive model
  ridge_model = ridge.fit(X_train, y_train)
  ridge_pred = ridge.predict(X_test)
  
  # Log model
  mlflow.sklearn.log_model(ridge_model, "log-regression-model")
  
  # Metrics
  rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
  print("Root Mean Squared Error: {}". format(rmse))
  mae = mean_absolute_error(y_test, ridge_pred)
  r_sq = r2_score(y_test, ridge_pred)
  print("R_squared: {}". format(r_sq))
  
  # Log metrics
  mlflow.log_metric("RMSE", rmse)
  mlflow.log_metric("R_2", r_sq)
  
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  
  print("Inside MLflow Run with run_id {}, experiment_id{}".format(runID, experimentID))